In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Enable inline plotting
%matplotlib inline

from IPython.display import display, Latex, clear_output
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [3]:
def query_gaia(name, cat='dr2', radius=0.003, return_all=False):

    """
    name : str
        name of source to search for
    cat : str
        GAIA 'dr1' or 'dr2'
    radius : float
        Search radius in degrees
    return_all : bool
        By default, returns Gaia source closest in position to searched object.
        By setting this to True, returns all sources found within search radius.
    """
    
    cat = cat.lower()

    sim = Simbad()
    sim.TIMEOUT = 300 # 5 minute timeout
    # Change URL to closer server
    sim.SIMBAD_URL = 'http://simbad.harvard.edu/simbad/sim-script'

    sim.remove_votable_fields('coordinates')

    if 'dr1' in cat:
        epoch = 'J2015'
    elif 'dr2' in cat:
        epoch = 'J2015.5'
    else:
        epoch = 'J2015.5'

    sim.add_votable_fields('ra(d;A;ICRS;{};2000)'.format(epoch),
                           'dec(d;D;ICRS;{};2000)'.format(epoch),
                           'flux(K)', 'sp', 'otype(V)', 'parallax')

    tbl = sim.query_object(name)

    ra_name1 = tbl.colnames[1]
    dec_name1 = tbl.colnames[2]

    ra_val1 = '{}'.format(tbl[ra_name1][0])
    dec_val1 = '{}'.format(tbl[dec_name1][0])

    if 'dr1' in cat:
        jstr = "SELECT TOP 10 source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,phot_g_mean_mag,phot_variable_flag \
            FROM gaiadr1.gaia_source  \
            WHERE CONTAINS(POINT('ICRS',gaiadr1.gaia_source.ra,gaiadr1.gaia_source.dec),\
                    CIRCLE('ICRS',{},{},{}))=1".format(ra_val1, dec_val1, radius)
    elif 'dr2' in cat:
        jstr = "SELECT TOP 10 source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,phot_g_mean_mag,bp_rp,\
            radial_velocity,radial_velocity_error,phot_variable_flag,teff_val,a_g_val \
            FROM gaiadr2.gaia_source \
            WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),\
                    CIRCLE('ICRS',{},{},{}))=1".format(ra_val1, dec_val1, radius)
        
    job1 = Gaia.launch_job(jstr)
    tbl1 = job1.get_results()
    
    if return_all:
        return tbl1

    if len(tbl1)>1:
        # Calculate Separations
        ra = tbl1.colnames[1]
        dec = tbl1.colnames[3]
        c1 = SkyCoord(tbl1[ra], tbl1[dec], frame='icrs',
                      obstime=epoch, equinox='J2000')
        c2 = SkyCoord(tbl[ra_name1], tbl[dec_name1], frame='icrs',
                      obstime=epoch, equinox='J2000')
        dist = c2.separation(c1)
        ind = dist == dist.min()
        tbl1 = tbl1[ind]

    return tbl1

In [4]:
sim = Simbad()
sim.TIMEOUT = 300 # 5 minute timeout
# Change URL to closer server
sim.SIMBAD_URL = 'http://simbad.harvard.edu/simbad/sim-script'

sim.add_votable_fields('ra(d;A;ICRS;J2000;2000)',
                       'dec(d;D;ICRS;J2000;2000)',
                       'flux(K)', 'sp', 'otype(V)', 'parallax')

names = ['HD10647', 'HD107146', 'HD181327', 'HD61005', 'HD32297']

In [5]:
darr = []
for name in names:
    tbl = sim.query_object(name)
    tbl1 = query_gaia(name, cat='dr1', radius=0.003, return_all=False)
    tbl2 = query_gaia(name, cat='dr2', radius=0.003, return_all=False)
        
    p0, _p0 = tbl[('PLX_VALUE','PLX_ERROR')][-1]

    if len(tbl1)==0:
        p1 = _p1 = np.nan
    else:    
        p1, _p1 = tbl1[('parallax','parallax_error')][-1]

    if len(tbl2)==0:
        p2 = _p2 = np.nan
    else:
        p2, _p2 = tbl2[('parallax','parallax_error')][-1]

    dist_arr = 1000 / np.array([p0,p1,p2])
    _dist_arr = np.array([_p0, _p1, _p2]) * dist_arr**2 / 1000
    
    d0, d1, d2 = dist_arr
    _d0, _d1, _d2 = _dist_arr
    odict = {'name':name, 'd0': d0, 'd1':d1, 'd2':d2,
             'd0_err':_d0, 'd1_err':_d1, 'd2_err':_d2,
             'p0':p0, 'p1':p1, 'p2':p2,
             'p0_err':_p0, 'p1_err':_p1, 'p2_err':_p2}
    darr.append(odict)


In [6]:
for odict in darr:
    if odict['d0']>0 or odict['d2']>0: print(odict['name'])
    if odict['d0']>0: 
        str1 = 'Simbad: {:.2f} +/- {:.4f} mas'.format(odict['p0'],odict['p0_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d0'], odict['d0_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d1']>0: 
        str1 = 'GAIA DR1: {:.2f} +/- {:.4f} mas'.format(odict['p1'],odict['p1_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d1'], odict['d1_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d2']>0: 
        str1 = 'GAIA DR2: {:.2f} +/- {:.4f} mas'.format(odict['p2'],odict['p2_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d2'], odict['d2_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d2']>0: print('')


HD10647
Simbad: 57.36 +/- 0.2500 mas (17.43 +/- 0.0760 pc)
GAIA DR1: 57.91 +/- 0.6434 mas (17.27 +/- 0.1919 pc)
GAIA DR2: 57.66 +/- 0.0737 mas (17.34 +/- 0.0222 pc)

HD107146
Simbad: 36.41 +/- 0.4000 mas (27.46 +/- 0.3017 pc)
GAIA DR1: 36.41 +/- 0.4036 mas (27.47 +/- 0.3045 pc)
GAIA DR2: 36.40 +/- 0.0418 mas (27.47 +/- 0.0315 pc)

HD181327
Simbad: 19.30 +/- 0.6500 mas (51.81 +/- 1.7450 pc)
GAIA DR1: 20.57 +/- 0.4453 mas (48.61 +/- 1.0523 pc)
GAIA DR2: 20.74 +/- 0.0576 mas (48.21 +/- 0.1338 pc)

HD61005
Simbad: 27.22 +/- 0.2600 mas (36.74 +/- 0.3509 pc)
GAIA DR1: 27.22 +/- 0.2592 mas (36.73 +/- 0.3498 pc)
GAIA DR2: 27.41 +/- 0.0322 mas (36.49 +/- 0.0428 pc)

HD32297
Simbad: 7.34 +/- 0.3400 mas (136.24 +/- 6.3108 pc)
GAIA DR1: 7.34 +/- 0.3409 mas (136.25 +/- 6.3279 pc)
GAIA DR2: 7.53 +/- 0.0599 mas (132.79 +/- 1.0564 pc)



In [ ]:
sim = Simbad()
sim.TIMEOUT = 300 # 5 minute timeout
# Change URL to closer server
sim.SIMBAD_URL = 'http://simbad.harvard.edu/simbad/sim-script'

sim.add_votable_fields('ra(d;A;ICRS;J2000;2000)',
                       'dec(d;D;ICRS;J2000;2000)',
                       'flux(K)', 'sp', 'otype(V)', 'parallax')


names = ['AA Tau', 'AB Aur', 'BP Tau', 
         'CX Tau', 'CY Tau', 'CW Tau', 'Coku Tau 4', 'CZ Tau', 
         'DG Tau', 'DM Tau', 'DR Tau', 'DQ Tau', 
         'FM Tau', 'GG Tau', 'GM Aur', 'HK Tau', 'HL Tau', 
         'IP Tau', 'IS Tau', 'RY Tau', 'T Tau',  'UX Tau',  'XZ Tau']
         
#names = ['AU Mic', 'HIP 17695', 'G 7-34', 'LP 776-25', 'G 161-71',
#         'FOMALHAUT C', 'AP Col', 'LP 944-20', '2MUCD 10320']

names = ['tau ceti', 'ax mic', 'GJ 380', 'sig Dra', '82 Eri', 'del Pav', 'GJ 892']
darr = []
for name in names:
    tbl = sim.query_object(name)
    tbl1 = query_gaia(name, cat='dr1', radius=0.003, return_all=False)
    tbl2 = query_gaia(name, cat='dr2', radius=0.003, return_all=False)
        
    p0, _p0 = tbl[('PLX_VALUE','PLX_ERROR')][-1]

    if len(tbl1)==0:
        p1 = _p1 = np.nan
    else:    
        p1, _p1 = tbl1[('parallax','parallax_error')][-1]

    if len(tbl2)==0:
        p2 = _p2 = np.nan
    else:
        p2, _p2 = tbl2[('parallax','parallax_error')][-1]

    dist_arr = 1000 / np.array([p0,p1,p2])
    _dist_arr = np.array([_p0, _p1, _p2]) * dist_arr**2 / 1000
    
    d0, d1, d2 = dist_arr
    _d0, _d1, _d2 = _dist_arr
    odict = {'name':name, 'd0': d0, 'd1':d1, 'd2':d2,
             'd0_err':_d0, 'd1_err':_d1, 'd2_err':_d2,
             'p0':p0, 'p1':p1, 'p2':p2,
             'p0_err':_p0, 'p1_err':_p1, 'p2_err':_p2}
    darr.append(odict)


for odict in darr:
    if odict['d0']>0 or odict['d2']>0: print(odict['name'])
    if odict['d0']>0: 
        str1 = 'Simbad: {:.2f} +/- {:.4f} mas'.format(odict['p0'],odict['p0_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d0'], odict['d0_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d1']>0: 
        str1 = 'GAIA DR1: {:.2f} +/- {:.4f} mas'.format(odict['p1'],odict['p1_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d1'], odict['d1_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d2']>0: 
        str1 = 'GAIA DR2: {:.2f} +/- {:.4f} mas'.format(odict['p2'],odict['p2_err'])
        str2 = '{:.2f} +/- {:.4f} pc'.format(odict['d2'], odict['d2_err'])
        print('{} ({})'.format(str1,str2))
    if odict['d2']>0: print('')


for odict in darr:
    if odict['d0']>0 or odict['d2']>0: print(odict['name'])
    if odict['d0']>0: print('Simbad: {:.2f} +/- {:.4f} pc'.format(odict['d0'], odict['d0_err']))
    if odict['d1']>0: print('GAIA DR1: {:.2f} +/- {:.4f} pc'.format(odict['d1'], odict['d1_err']))
    if odict['d2']>0: print('GAIA DR2: {:.2f} +/- {:.4f} pc'.format(odict['d2'], odict['d2_err']))
    if odict['d2']>0: print('')

for odict in darr:
    if odict['d0']>0 or odict['d2']>0: print(odict['name'])
    if odict['d0']>0: print('Simbad: {:.2f} +/- {:.4f} mas'.format(odict['p0'],odict['p0_err']))
    if odict['d1']>0: print('GAIA DR1: {:.2f} +/- {:.4f} mas'.format(odict['p1'],odict['p1_err']))
    if odict['d2']>0: print('GAIA DR2: {:.2f} +/- {:.4f} mas'.format(odict['p2'],odict['p2_err']))
    if odict['d2']>0: print('')



